In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Fork-Join queueing network
model = Network('model')

In [3]:
# Create network 
source = Source(model, 'Source')

queue1 = Queue(model, 'Queue1', SchedStrategy.PS)
queue2 = Queue(model, 'Queue2', SchedStrategy.PS)

fork = Fork(model, 'Fork')
fork.set_tasks_per_link(2)
join = Join(model, 'Join', fork)

sink = Sink(model, 'Sink')

In [4]:
# Create job classes
jobclass1 = OpenClass(model, 'class1')
jobclass2 = OpenClass(model, 'class2')

In [5]:
#Set arrival and service processes
# Class 1
source.set_arrival(jobclass1, Exp(0.25))
queue1.set_service(jobclass1, Exp(1.0))
queue2.set_service(jobclass1, Exp(0.75))
# Class 2
source.set_arrival(jobclass2, Exp(0.25))
queue1.set_service(jobclass2, Immediate())
queue2.set_service(jobclass2, Exp(2.0))

In [6]:
# Set routing matrix
P = model.init_routing_matrix() # Class 1 routing
P.set(jobclass1, jobclass1, source, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, join, 1.0)
P.set(jobclass1, jobclass1, join, sink, 1.0) # Class 2 routing
P.set(jobclass2, jobclass2, source, fork, 1.0)
P.set(jobclass2, jobclass2, fork, queue1, 1.0)
P.set(jobclass2, jobclass2, fork, queue2, 1.0)
P.set(jobclass2, jobclass2, queue1, join, 1.0)
P.set(jobclass2, jobclass2, queue2, join, 1.0)
P.set(jobclass2, jobclass2, join, sink, 1.0)
model.link(P)

In [7]:
# Solve with multiple methods
solvers = [ SolverJMT(model, seed=23000), SolverMVA(model)]
for i, solver in enumerate(solvers):
    print(f'SOLVER {i+1}: {solver.get_name()}')
    avgTable = solver.get_avg_table()

SOLVER 1: SolverJMT
JMT Model: /tmp/workspace/jsim/15479674282163109897/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 1.202283s.
  Station JobClass     QLen    Util    RespT   ResidT    ArvR    Tput
0  Source   class1   0.0000  0.0000   0.0000   0.0000  0.0000  0.2510
1  Source   class2   0.0000  0.0000   0.0000   0.0000  0.0000  0.2477
2  Queue1   class1   1.3998  0.5127   3.0043   1.5021  0.5209  0.5209
3  Queue1   class2   0.0000  0.0000   0.0000   0.0000  0.5004  0.5004
4  Queue2   class1  11.3398  0.6998  20.6083  10.3042  0.5209  0.5115
5  Queue2   class2   4.1457  0.2507   8.5604   4.2802  0.5004  0.5061
6    Join   class1  20.0231  0.0000  19.2747  19.2747  1.0307  0.2507
7    Join   class2   8.1158  0.0000   8.1805   8.1805  1.0108  0.2480
SOLVER 2: SolverMVA
MVA analysis [method: default(egflin), lang: java, env: 21.0.8] completed in 0.151226s.
  Station JobClass     QLen    Util       RespT   ResidT  ArvR  Tput
0  Source   class1   0.0000  0.0000  0